<a href="https://colab.research.google.com/github/cesong2/Display-Sensor/blob/CGKim/teamproject2%EB%B2%88%EC%A7%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np

In [54]:
from datetime import timedelta
import datetime
import numpy as np
from datetime import datetime
import time

In [5]:
#Load the data
telemetry_df = pd.read_csv("/content/PdM_telemetry.csv")
errors_df = pd.read_csv("/content/PdM_errors.csv")
maint_df = pd.read_csv("/content/PdM_maint.csv")
failures_df = pd.read_csv("/content/PdM_failures.csv")
machines_df = pd.read_csv("/content/PdM_machines.csv")
machines_age = pd.DataFrame(machines_df[['machineID','age']]).set_index('machineID')
machines_errors = pd.DataFrame(errors_df.groupby('machineID')['errorID'].count())
machines_maint = pd.DataFrame(maint_df.groupby('machineID')['comp'].count())
machines_failures = pd.DataFrame(failures_df.groupby('machineID')['failure'].count())

#컴포넌트 교체주기 확인$2

In [6]:
#maint 데이터파일 0과 1로 정리해주기
import numpy as np
comp_rep = pd.get_dummies(maint_df.set_index('datetime')).reset_index()
comp_rep.columns = ['datetime', 'machineID','comp1', 'comp2', 'comp3', 'comp4']
comp_rep = telemetry_df[['datetime', 'machineID']].merge(comp_rep,on=['datetime','machineID'],how='outer').fillna(0).sort_values(by=['machineID', 'datetime'])

In [7]:
#maint 데이터파일 시간대별로 채우기
components = ['comp1', 'comp2', 'comp3', 'comp4']
for comp in components:
    comp_rep.loc[comp_rep[comp] < 1, comp] = None
    comp_rep.loc[-comp_rep[comp].isnull(),
                 comp] = comp_rep.loc[-comp_rep[comp].isnull(), 'datetime']
    comp_rep[comp] = comp_rep[comp].fillna(method='ffill')

##몇번 교체되었는지 component별 확인

In [8]:
#컴포넌트별 모든 시간대 정리
comp1_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp1'])
comp2_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp2'])
comp3_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp3'])
comp4_t = pd.DataFrame(comp_rep, columns = ['datetime','machineID','comp4'])

In [9]:
#컴포넌트 시간대별 중복 데이터 정리 (586799데이터 中)
comp1_rep = comp1_t.drop_duplicates(['comp1']) #303데이터열
comp2_rep = comp2_t.drop_duplicates(['comp2']) #317데이터열
comp3_rep = comp3_t.drop_duplicates(['comp3']) #311데이터열
comp4_rep = comp4_t.drop_duplicates(['comp4']) #314데이터열

In [17]:
print(f'comp1은 {len(comp1_rep)}번 교체, comp2은 {len(comp2_rep)}번 교체, comp3은 {len(comp3_rep)}번 교체, comp4은 {len(comp4_rep)}번 교체,')

comp1은 303번 교체, comp2은 317번 교체, comp3은 311번 교체, comp4은 314번 교체,


In [18]:
comp1 = comp1_rep.set_index('datetime').reset_index()
comp2 = comp2_rep.set_index('datetime').reset_index()
comp3 = comp3_rep.set_index('datetime').reset_index()
comp4 = comp4_rep.set_index('datetime').reset_index()

In [19]:
#결측치 제거 전 (Nan값제거)
comp1
comp2
comp3
comp4
#결측치 제거 후 (Nan값제거)
comp1_n = comp1.dropna(axis=0) #결측치 1개 삭제
comp2_n = comp2.dropna(axis=0) #결측치 0개 삭제
comp3_n = comp3.dropna(axis=0) #결측치 1개 삭제
comp4_n = comp4.dropna(axis=0) #결측치 1개 삭제
#순서 재배치
comp1_n = comp1_n.set_index('datetime').reset_index()
comp2_n = comp2_n.set_index('datetime').reset_index()
comp3_n = comp3_n.set_index('datetime').reset_index()
comp4_n = comp4_n.set_index('datetime').reset_index()

In [84]:
failures_df.count()

datetime     761
machineID    761
failure      761
dtype: int64

In [83]:
maint_df.count()

datetime     3286
machineID    3286
comp         3286
dtype: int64

#고장에 의한 교체 찾기$1

In [115]:
df = pd.DataFrame(columns = {'machineID','failures_maint','datetime'})

In [116]:
n = maint_df.merge(failures_df, on = ('datetime','machineID'),how = 'inner')
for i in range(len(n)):
  if n['comp'][i] == n['failure'][i]: #겹치는애들

    df.loc[i] = [n['failure'][i],n['datetime'][i],n['machineID'][i]]
  else:#겹치지 않는애들 (제거 요망)
    pass

In [ ]:
n #1104 rows

In [128]:
df #743 rows
df1 = df.set_index('datetime').reset_index() #숫자 재정렬
df1

,datetime,failures_maint,machineID
0,2015-01-05 06:00:00,comp4,1
1,2015-03-06 06:00:00,comp1,1
2,2015-04-20 06:00:00,comp2,1
3,2015-06-19 06:00:00,comp4,1
4,2015-09-02 06:00:00,comp4,1
...,...,...,...
738,2015-11-29 06:00:00,comp3,99
739,2015-12-14 06:00:00,comp4,99
740,2015-02-12 06:00:00,comp1,100
741,2015-09-10 06:00:00,comp1,100


In [125]:
df_comp1 = df1[df1['failures_maint']== 'comp1'] #failures_maint별 정리
df_comp2 = df1[df1['failures_maint']== 'comp2']
df_comp3 = df1[df1['failures_maint']== 'comp3']
df_comp4 = df1[df1['failures_maint']== 'comp4']
df_com1 = df_comp1.set_index('datetime').reset_index()#숫자 재정렬
df_com2 = df_comp2.set_index('datetime').reset_index()
df_com3 = df_comp3.set_index('datetime').reset_index()
df_com4 = df_comp4.set_index('datetime').reset_index()

In [143]:
df_com1.head(5)

,datetime,failures_maint,machineID,time_diff
0,2015-03-06 06:00:00,comp1,1,13.0
1,2015-03-19 06:00:00,comp1,2,NaN
2,2015-02-06 06:00:00,comp1,3,10.0
3,2015-02-16 06:00:00,comp1,4,195.0
4,2015-08-30 06:00:00,comp1,4,NaN


In [91]:
len(lin) #failures_df와 maint_df데이터 합쳤을때 총 761 ->743데이터로 변환

743

In [171]:
#datetime 을 이용한 교체주기 확인
fail_com = []
n = 1
for i in range(len(df_com1)-1):
  if n != range(len(df_com1)):
    str_datetime1 = df_com1["datetime"][n]
    str_datetime2 = df_com1["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com.append(new.days)
    n += 1
  else:
    break
fail_com1 = []
n1 = 1
for i in range(len(df_com2)-1):
  if n1 != range(len(df_com2)):
    str_datetime1 = df_com2["datetime"][n1]
    str_datetime2 = df_com2["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com1.append(new.days)
    n1 += 1
  else:
    break
fail_com2 = []
n2 = 1
for i in range(len(df_com3)-1):
  if n2 != range(len(df_com3)):
    str_datetime1 = df_com3["datetime"][n2]
    str_datetime2 = df_com3["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com2.append(new.days)
    n2 += 1
  else:
    break
fail_com3 = []
n3 = 1
for i in range(len(df_com4)-1):
  if n3 != range(len(df_com4)):
    str_datetime1 = df_com4["datetime"][n3]
    str_datetime2 = df_com4["datetime"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    fail_com3.append(new.days)
    n3 += 1
  else:
    break

In [177]:
#리스트 안의 음수를 none으로 바꾸기 #def로 바꾸기
fail_comp =[]
for i in range(len(fail_com)):
  if fail_com[i] > 0:
    fail_comp.append(fail_com[i])
  else:
    fail_comp.append(None)
fail_comp1 =[]
for i in range(len(fail_com1)):
  if fail_com1[i] > 0:
    fail_comp1.append(fail_com1[i])
  else:
    fail_comp1.append(None)
fail_comp2 =[]
for i in range(len(fail_com2)):
  if fail_com2[i] > 0:
    fail_comp2.append(fail_com2[i])
  else:
    fail_comp2.append(None)
fail_comp3 =[]
for i in range(len(fail_com3)):
  if fail_com3[i] > 0:
    fail_comp3.append(fail_com3[i])
  else:
    fail_comp3.append(None)

In [181]:
#마지막 한 열은 다음 계산할것이 없으니 None값 하나를 추가해줘서 index개수 맞춰주기
fail_comp.append(None)
fail_comp1.append(None) 
fail_comp2.append(None) 
fail_comp3.append(None) 

In [182]:
#기존데이터에 dataframe으로 추가해주기
df_com1['time_diff'] = fail_comp
df_com2['time_diff'] = fail_comp1
df_com3['time_diff'] = fail_comp2
df_com4['time_diff'] = fail_comp3

In [183]:
#def 로 바꾸기
li_co = []#한개씩가져오기
li1_co = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co:
    li_co.append(df_com1['machineID'][i])
  else:
    li1_co.append(df_com1['machineID'][i])
li_co1 = []#한개씩가져오기
li1_co1 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co1:
    li_co1.append(df_com1['machineID'][i])
  else:
    li1_co1.append(df_com1['machineID'][i])
li_co2 = []#한개씩가져오기
li1_co2 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co2:
    li_co2.append(df_com1['machineID'][i])
  else:
    li1_co2.append(df_com1['machineID'][i])
li_co3 = []#한개씩가져오기
li1_co3 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(df_com1)):
  if df_com1['machineID'][i] not in li_co3:
    li_co3.append(df_com1['machineID'][i])
  else:
    li1_co3.append(df_com1['machineID'][i])
  

In [184]:
#중복처리
li1_co=set(li1_co)
li1_co1=set(li1_co1)
li1_co2=set(li1_co2)
li1_co3=set(li1_co3)

In [187]:
#한개씩 있는 머신아이디 추출
fail_c = []
for i in range(len(li_co)):
  if li_co[i] not in li1_co:
    fail_c.append(li_co[i])
  else:
    pass
fail_c1 = []
for i in range(len(li_co1)):
  if li_co1[i] not in li1_co1:
    fail_c1.append(li_co1[i])
  else:
    pass
fail_c2 = []
for i in range(len(li_co2)):
  if li_co2[i] not in li1_co2:
    fail_c2.append(li_co2[i])
  else:
    pass
fail_c3 = []
for i in range(len(li_co3)):
  if li_co3[i] not in li1_co3:
    fail_c3.append(li_co3[i])
  else:
    pass

In [188]:
#한개씩 있는 machineID의 time_diff를 nan값으로 처리
for i in range(len(fail_c)):
  df_com1.loc[df_com1['machineID']==fail_c[i],'time_diff'] = None
for i in range(len(fail_c1)):
  df_com2.loc[df_com2['machineID']==fail_c1[i],'time_diff'] = None
for i in range(len(fail_c2)):
  df_com3.loc[df_com3['machineID']==fail_c2[i],'time_diff'] = None
for i in range(len(fail_c3)):
  df_com4.loc[df_com4['machineID']==fail_c3[i],'time_diff'] = None

In [191]:
ans1 = df_com1['time_diff'].mean().round(2)
ans2 = df_com2['time_diff'].mean().round(2)
ans3= df_com3['time_diff'].mean().round(2)
ans4= df_com4['time_diff'].mean().round(2)
print(f'고장난 컴포넌트 1의 평균 교체주기 (day)는 {ans1}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {ans2}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {ans3}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {ans4}일입니다')

고장난 컴포넌트 1의 평균 교체주기 (day)는 107.43일입니다,   컴포넌트 2의 평균 교체주기 (day)는 89.36일입니다,    컴포넌트 3의 평균 교체주기 (day)는 87.53일입니다,    컴포넌트 4의 평균 교체주기 (day)는 95.39일입니다


#datetime을 이용한 교체주기 확인 $2

In [ ]:
#나중에 def 로 수정해주기

In [153]:
#datetime 을 이용한 교체주기 확인
comp1_re = []
n = 1
for i in range(len(comp1_n)-1):
  if n != range(len(comp1_n)):
    str_datetime1 = comp1_n["comp1"][n]
    str_datetime2 = comp1_n["comp1"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp1_re.append(new.days)
    n += 1
  else:
    break
comp2_re = []
n1 = 1
for i in range(len(comp2_n)-1):
  if n1 != range(len(comp2_n)):
    str_datetime1 = comp2_n["comp2"][n1]
    str_datetime2 = comp2_n["comp2"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp2_re.append(new.days)
    n1 += 1
  else:
    break
comp3_re = []
n2 = 1
for i in range(len(comp3_n)-1):
  if n2 != range(len(comp3_n)):
    str_datetime1 = comp3_n["comp3"][n2]
    str_datetime2 = comp3_n["comp3"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp3_re.append(new.days)
    n2 += 1
  else:
    break
comp4_re = []
n3 = 1
for i in range(len(comp4_n)-1):
  if n3 != range(len(comp4_n)):
    str_datetime1 = comp4_n["comp4"][n3]
    str_datetime2 = comp4_n["comp4"][i]
    currdate1 = datetime.strptime(str_datetime1, '%Y-%m-%d %H:%M:%S')
    currdate2 = datetime.strptime(str_datetime2, '%Y-%m-%d %H:%M:%S')
    new = currdate1- currdate2
    comp4_re.append(new.days)
    n3 += 1
  else:
    break

In [154]:
 # 음수인 데이터는 machineID 가 바뀌면서 생기는 데이터로 사실상 의미가없음.
len(comp1_re),len(comp2_re),len(comp3_re),len(comp4_re)

(301, 316, 309, 312)

#리스트안의 음수를 nan 처리하고, 기존데이터에 dataframe으로 추가 $2

In [155]:
#리스트 안의 음수를 none으로 바꾸기 #def로 바꾸기
comp1_re1 =[]
for i in range(len(comp1_re)):
  if comp1_re[i] > 0:
    comp1_re1.append(comp1_re[i])
  else:
    comp1_re1.append(None)
  
comp2_re1 =[]
for i in range(len(comp2_re)):
  if comp2_re[i] > 0:
    comp2_re1.append(comp2_re[i])
  else:
    comp2_re1.append(None)
  
comp3_re1 =[]
for i in range(len(comp3_re)):
  if comp3_re[i] > 0:
    comp3_re1.append(comp3_re[i])
  else:
    comp3_re1.append(None)
  
comp4_re1 =[]
for i in range(len(comp4_re)):
  if comp4_re[i] > 0:
    comp4_re1.append(comp4_re[i])
  else:
    comp4_re1.append(None)

In [156]:
#마지막 한 열은 다음 계산할것이 없으니 None값 하나를 추가해줘서 index개수 맞춰주기
comp1_re1.append(None) 
comp2_re1.append(None) 
comp3_re1.append(None) 
comp4_re1.append(None) 

In [157]:
#기존데이터에 dataframe으로 추가해주기
comp1_n['time_diff'] = comp1_re1
comp2_n['time_diff'] = comp2_re1
comp3_n['time_diff'] = comp3_re1
comp4_n['time_diff'] = comp4_re1

In [158]:
comp2_n.tail(10)

,datetime,machineID,comp2,time_diff
307,2015-03-24 06:00:00,89,2015-03-24 06:00:00,165.0
308,2015-09-05 06:00:00,89,2015-09-05 06:00:00,15.0
309,2015-09-20 06:00:00,89,2015-09-20 06:00:00,NaN
310,2015-03-21 06:00:00,90,2015-03-21 06:00:00,57.0
311,2015-05-17 06:00:00,91,2015-05-17 06:00:00,NaN
312,2015-01-08 06:00:00,94,2015-01-08 06:00:00,NaN
313,2015-01-03 06:00:00,96,2015-01-03 06:00:00,126.0
314,2015-05-09 06:00:00,97,2015-05-09 06:00:00,NaN
315,2015-04-03 06:00:00,99,2015-04-03 06:00:00,115.0
316,2015-07-27 06:00:00,100,2015-07-27 06:00:00,NaN


In [159]:
#machineID가 한개일경우 time_diff 는 nan값이 되도록 전처리하기

In [160]:
#def 로 바꾸기
li = []#한개씩가져오기
li1 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp1_n)):
  if comp1_n['machineID'][i] not in li:
    li.append(comp1_n['machineID'][i])
  else:
    li1.append(comp1_n['machineID'][i])


li2 = []#한개씩가져오기
li3 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp2_n)):
  if comp2_n['machineID'][i] not in li2:
    li2.append(comp2_n['machineID'][i])
  else:
    li3.append(comp2_n['machineID'][i])


li4 = []#한개씩가져오기
li5 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp3_n)):
  if comp3_n['machineID'][i] not in li4:
    li4.append(comp3_n['machineID'][i])
  else:
    li5.append(comp3_n['machineID'][i])


li6 = []#한개씩가져오기
li7 = []#최소 한개 이상인 데이터 머신아이디 뽑아오기
for i in range(len(comp4_n)):
  if comp4_n['machineID'][i] not in li6:
    li6.append(comp4_n['machineID'][i])
  else:
    li7.append(comp4_n['machineID'][i])

In [161]:
#중복처리
li1=set(li1)
li3=set(li3)
li5=set(li5)
li7=set(li7)

In [162]:
#한개씩 있는 머신아이디 추출
nli1 = []
for i in range(len(li)):
  if li[i] not in li1:
    nli1.append(li[i])
  else:
    pass

nli2 = []
for i in range(len(li2)):
  if li2[i] not in li3:
    nli2.append(li2[i])
  else:
    pass

nli3 = []
for i in range(len(li4)):
  if li4[i] not in li5:
    nli3.append(li4[i])
  else:
    pass

nli4 = []
for i in range(len(li6)):
  if li6[i] not in li7:
    nli4.append(li6[i])
  else:
    pass

In [163]:
 #한개씩 있는 머신아이디 추출
nli1
nli2
nli3
nli4

[15, 34, 43, 51, 52, 60, 76, 80, 82, 86, 88, 90, 94, 97, 100]

In [164]:
#nli를 이용하여 한개씩 있는 machineID의 time_diff를 nan값으로 처리
for i in range(len(nli1)):
  comp1_n.loc[comp1_n['machineID']==nli1[i],'time_diff'] = None
for i in range(len(nli2)):
  comp2_n.loc[comp2_n['machineID']==nli2[i],'time_diff'] = None
for i in range(len(nli3)):
  comp3_n.loc[comp3_n['machineID']==nli3[i],'time_diff'] = None
for i in range(len(nli4)):
  comp4_n.loc[comp4_n['machineID']==nli4[i],'time_diff'] = None

In [165]:
#데이터 개수 확인
len(comp1_n),len(comp2_n),len(comp3_n),len(comp4_n)

(302, 317, 310, 313)

##교체 주기 확인 $2

In [166]:
#day기준
comp1_n['time_diff'].describe()
comp2_n['time_diff'].describe()
comp3_n['time_diff'].describe()
comp4_n['time_diff'].describe()
comp_mean1=comp1_n['time_diff'].mean().round(3)
comp_mean2=comp2_n['time_diff'].mean().round(3)
comp_mean3=comp3_n['time_diff'].mean().round(3)
comp_mean4=comp4_n['time_diff'].mean().round(3)
print(f'컴포넌트 1의 평균 교체주기 (day)는 {comp_mean1}일입니다,   컴포넌트 2의 평균 교체주기 (day)는 {comp_mean2}일입니다,    컴포넌트 3의 평균 교체주기 (day)는 {comp_mean3}일입니다,    컴포넌트 4의 평균 교체주기 (day)는 {comp_mean4}일입니다')

컴포넌트 1의 평균 교체주기 (day)는 73.655일입니다,   컴포넌트 2의 평균 교체주기 (day)는 63.177일입니다,    컴포넌트 3의 평균 교체주기 (day)는 76.194일입니다,    컴포넌트 4의 평균 교체주기 (day)는 69.667일입니다
